In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

In [ ]:
from azureml.core import Workspace, Dataset

subscription_id = '###subscription-id'
resource_group = '###resource-group'
workspace_name = '###ml-workspace'

ws = Workspace(subscription_id, resource_group, workspace_name)

exp = ws.experiments['###experiment-name']

e = list(exp.get_runs())[0]
print(e)

In [ ]:
aml_run_details = e.get_details()
aml_run_details

In [ ]:
import json

aml_properties = aml_run_details['properties']
aml_settings = json.loads(aml_properties['AMLSettingsJsonString'])
aml_settings


In [ ]:
print(aml_properties['DataPrepJsonString'])
datasets = json.loads(aml_properties['DataPrepJsonString'].replace("\\",""))
training_ds_id =  datasets['training_data']['datasetId']
training_ds = Dataset.get_by_id(ws, training_ds_id)
print(training_ds)

In [ ]:
#Override parameters for new run
aml_settings['compute_target'] = '##Compute-target'
aml_settings['blacklist_algos'] = ['TensorFlowLinearClassifier', 'TensorFlowDNN', 'LinearSVM', 'AveragedPerceptronClassifier']


automl_config = AutoMLConfig(task=aml_settings['task_type'],
                            training_data = training_ds,
                             **aml_settings
                            )


# Uncomment below to submit experiment 

In [ ]:
#remote_run = exp.submit(automl_config, show_output = False)

In [ ]:
#remote_run.wait_for_completion()

# Change feature settings for next AutoML experiment

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig

featurization_config = FeaturizationConfig()

featurization_config.add_column_purpose('column_1', 'Categorical')
featurization_config.add_column_purpose('column_2','Numeric')

In [ ]:
#Override parameters for new run
aml_settings['compute_target'] = '###Compute-Target'
aml_settings['blacklist_algos'] = ['TensorFlowLinearClassifier', 'TensorFlowDNN', 'LinearSVM', 'AveragedPerceptronClassifier']
aml_settings['featurization'] = featurization_config


automl_config = AutoMLConfig(task=aml_settings['task_type'],
                            training_data = training_ds,
                             **aml_settings
                            )
